<a href="https://colab.research.google.com/github/arthurnonaka/goalpost_detection/blob/master/goalpost_detection_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TREINAMENTO DE REDES NEURAIS CONVOLUCIONAIS PARA DETECÇÃO DE TRAVES DE FUTEBOL

O objetivo desse Notebook é oferecer ao usuário uma forma prática e fácil de treinar uma rede neural convolucional para detecção de objetos, o intuito é que com algumas alterações e inserindo as imagens juntamente com as labels o treinamento seja executado em um só lugar.

##Configuração e instalação dos pacotes necessários.

###Check GPU configurations working properly

  A etapa a baixo configura o sistema para usar GPU e checa se está funcionando, mas antes disso deve-se conectar ao GPU, para isso basta clicar em "Runtime", depois "Change Runtime Type", e selecionar Python3 e GPU.
  
  Depois é só clicar em "Connect" no canto superior direito da tela.

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
 
tf.compat.v1.disable_eager_execution()
with tf.device('/gpu:0'):
  random_image_gpu = tf.compat.v1.random_normal((100, 100, 100, 3))
  net_gpu = tf.compat.v1.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.compat.v1.reduce_sum(net_gpu)
   
sess = tf.compat.v1.Session(config=config)
 
try:
  sess.run(tf.compat.v1.global_variables_initializer())
  print('ok')
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

TensorFlow 1.x selected.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
ok


###Connect to the Google Drive

In [ ]:
from os.path import join
from google.colab import drive
 
ROOT = "/content/drive"
drive.mount(ROOT)
 
PROJ = "My Drive/goalpost_detection_master" # This is a custom path.
PROJECT_PATH = join(ROOT, PROJ)
 
%cd ~/content
%cd drive/My Drive/goalpost_detection_master


Mounted at /content/drive
[Errno 2] No such file or directory: '/root/content'
/content
/content/drive/My Drive/goalpost_detection_master


Aqui estaremos conectando o notebook do Google Colab ao nosso Drive, para que ele tenha acesso aos nossos arquivos e parâmetros de configuração. Será necessário um código de permissão que pode ser obtido no link que vai ser gerado.

###Clone the Tensorflow models repository (ONLY ONCE!)

A célula a baixo baixa o repositório do Tensorflow com as bibliotecas necessárias para treinar a rede neural na plataforma, caso já tenha rodado uma vez não é necessário rodar de novo.

In [ ]:
!git clone https://github.com/dodandeniya/TFmodels.git

fatal: destination path 'TFmodels' already exists and is not an empty directory.


###Compile the library and run unit test


Nessa etapa nós vamos compilar e rodar um teste para saber se a classe object_detection do tensorflow está funcionando. Essa biblioteca vai ser necessária para o treinamento da nossa rede.

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/TFmodels/research
!pwd
!protoc object_detection/protos/*.proto --python_out=.


/root
/content
/content/drive/My Drive/goalpost_detection_master/TFmodels/research
/content/drive/My Drive/goalpost_detection_master/TFmodels/research


In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/goalpost_detection_master/TFmodels/research/:/content/drive/My Drive/goalpost_detection_master/TFmodels/research/slim/'
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/goalpost_detection_master/TFmodels/research/:/content/drive/My Drive/goalpost_detection_master/TFmodels/research/object_detection/'
!export PYTHONPATH=PYTHONPATH:':/content/drive/My Drive/goalpost_detection_master/TFmodels/research/:/content/drive/My Drive/goalpost_detection_master/TFmodels/research/slim/'

!python TFmodels/research/object_detection/builders/model_builder_test.py

# Change the directory path
 
%cd TFmodels/research/object_detection

/root
/content
/content/drive/My Drive/goalpost_detection_master
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Running tests under Python 3.7.11: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_without_matmul
[       OK ] ModelBuilderTest.test_create

###Importing necessary headers

Na célula a baixo iremos testar se todos as bibliotecas necessárias estão instaladas corretamente, se estiverem a célula não apresentará nenhum erro.

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/
%cd TFmodels/research/slim/
import nets
import deployment

%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/
%cd TFmodels/research/
import functools
import json
import os
import tensorflow as tf

from object_detection.builders import dataset_builder
from object_detection.builders import graph_rewriter_builder
from object_detection.builders import model_builder
from object_detection.legacy import trainer
from object_detection.utils import config_util

/root
/content
/content/drive/My Drive/goalpost_detection_master
/content/drive/My Drive/goalpost_detection_master/TFmodels/research/slim
/root
/content
/content/drive/My Drive/goalpost_detection_master
/content/drive/My Drive/goalpost_detection_master/TFmodels/research
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





##Model preparation (ONLY ONCE!)

Aqui nós faremos a preparação do modelo, que envolve algumas etapas básicas:

**1 - Definir o train_set e o test_set**

**2 - Separar as imagens e organizar o nome (Ex: ball_001.png)**

**3 - Preparar as labels**

**4 - Gerar o trainval.txt (./annotations/traival_generate.py -> P/ rodar: python trainval_generate.py)**

**5 - Preparar o label_map.pbtxt (./object_detection/data/label_map.pbtxt)**

**6 - Gerar os arquivos .record (./create_goal_tf_record.py -> P/ rodar: python create_goal_tf_record.py --label_map_path ./object_detection/data/goalpost_label_map.pbtxt --data_dir ~/goal_detection-master/ --output_dir ./object_detection/data/)**

**7 - Baixar o modelo pré treinado (https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)**

**8 - Criar um modelo de configuração de acordo com o pré treinado (https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs)**

**9 - Configurar o modelo de configuração**

Vamos levar em consideração que os passos 1, 2 e 3 já foram feitos anteriormente, caso não tenham sido realizados basta procurar rapidamente na internet, que existem vários tutoriais ensinando.

Passemos para o passo 4!

###Creating trainval.txt

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/annotations/

!python3 trainval_generate.py

/root
/content
/content/drive/My Drive/goalpost_detection_master/annotations


###Writing label_map.pbtxt file

Esse arquivo contém as classes nas quais a nossa rede vai ser treinada, ela deve seguir esse formato.

Caso tenha mais de uma classe basta adicionar no arquivo outro item com o mesmo nome das labels

OBS: o caminho do arquivo deve ser:

./object_detection/data/label_map.pbtxt

In [ ]:
#Modelo do arquivo label_map.pbtxt (Não é para compilar)
#item {
#
#  id: 1
#  
#  name: 'goalpost'
#  
#}

###.record files generate

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/
!python3 create_goal_tf_record.py --label_map_path ./object_detection/data/goalpost_label_map.pbtxt --data_dir ./ --output_dir ./object_detection/data/


/root
/content
/content/drive/My Drive/goalpost_detection_master


W0726 18:43:36.276252 140206769600384 module_wrapper.py:139] From /content/drive/My Drive/goalpost_detection_master/TFmodels/research/object_detection/utils/label_map_util.py:132: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

I0726 18:43:36.278232 140206769600384 create_goal_tf_record.py:188] Reading from Pet dataset.
I0726 18:43:36.280526 140206769600384 create_goal_tf_record.py:203] 414 training and 5 validation examples.

W0726 18:43:36.280730 140206769600384 module_wrapper.py:139] From create_goal_tf_record.py:163: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

I0726 18:43:36.284476 140206769600384 create_goal_tf_record.py:166] On image 0 of 414
/content/drive/My Drive/goalpost_detection_master/TFmodels/research/object_detection/utils/dataset_util.py:75: FutureWarning: The behavior of this method will change in future versions. Use specif

##Executing object detection training script

Chegamos na parte prática do notebook que envolve treinar a rede neural e depois gerar o frozen graph para testar os resultados. 
Abaixo se encontram algumas informações da fonte do script utilizado para realizar o treinamento, com os parâmetros utilizados e a forma de compilar.

**Copyright 2017 The TensorFlow Authors. All Rights Reserved.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at:**

    **http://www.apache.org/licenses/LICENSE-2.0**
    
**Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.**

Training executable for detection models.

This executable is used to train DetectionModels. There are two ways of
configuring the training job:

1) A single pipeline_pb2.TrainEvalPipelineConfig configuration file
can be specified by --pipeline_config_path.

Example usage:
    ./train \
        --logtostderr \
        --train_dir=path/to/train_dir \
        --pipeline_config_path=pipeline_config.pbtxt

2) Three configuration files can be provided: a model_pb2.DetectionModel
configuration file to define what type of DetectionModel is being trained, an
input_reader_pb2.InputReader file to specify what training data will be used and
a train_pb2.TrainConfig file to configure training parameters.

Example usage:
    ./train \
        --logtostderr \
        --train_dir=path/to/train_dir \
        --model_config_path=model_config.pbtxt \
        --train_config_path=train_config.pbtxt \
        --input_config_path=train_input_config.pbtxt

###Training the Neural Network

Esse passo pode demorar algumas horas ou até dias dependendo dos parâmetros escolhidos.
DICA: caso queira um treinamento mais rápido deve-se diminuir o Batch size, e o número de steps, esses dois parâmetros estão localizados em:
"./object_detection/data/ssd_inception_v2_pets.config"

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/
!python3 train.py --logtostderr --train_dir ./object_detection/data --pipeline_config_path ./object_detection/data/ssd_inception_v2_coco.config

A saída de streaming foi truncada nas últimas 5000 linhas.
INFO:tensorflow:global step 37522: loss = 2.2643 (0.738 sec/step)
I0728 01:36:22.135679 140063793756032 learning.py:507] global step 37522: loss = 2.2643 (0.738 sec/step)
INFO:tensorflow:global step 37523: loss = 2.2551 (0.785 sec/step)
I0728 01:36:22.922026 140063793756032 learning.py:507] global step 37523: loss = 2.2551 (0.785 sec/step)
INFO:tensorflow:global step 37524: loss = 2.7929 (0.711 sec/step)
I0728 01:36:23.634410 140063793756032 learning.py:507] global step 37524: loss = 2.7929 (0.711 sec/step)
INFO:tensorflow:global step 37525: loss = 1.7875 (0.709 sec/step)
I0728 01:36:24.345069 140063793756032 learning.py:507] global step 37525: loss = 1.7875 (0.709 sec/step)
INFO:tensorflow:global step 37526: loss = 2.3670 (0.746 sec/step)
I0728 01:36:25.092813 140063793756032 learning.py:507] global step 37526: loss = 2.3670 (0.746 sec/step)
INFO:tensorflow:global step 37527: loss = 2.7212 (0.748 sec/step)
I0728 01:36:25.84201

Como foi dito anteriormente, esse processo pode demorar horas, e infelizmente o Google Colab corta a conexão caso haja uma inatividade de 90min. Para evitar esse tipo de transtorno pode-se:
 
 - Modo trabalhoso: Antes do tempo de inatividade ser atingido, mexer no Notebook (inviável para treinamentos longos de várias horas);
 - Modo cotoco: Programar um código em Python ou qualquer outra linguagem que mexa o mouse ou role a tela pra baixo e pra cima automaticamente depois de certo tempo.

###Exporting Frozen Graph

Depois de treinada é necessário salvar os parâmetros encontrados para testar nas imagens. Esse trabalho é feito utilizando o código export_inference_graph.py

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/
!python3 export_inference_graph.py --input_type image_tensor --pipeline_config_path ./object_detection/data/ssd_inception_v2_coco.config --trained_checkpoint_prefix ./object_detection/data/model.ckpt-40000 --output_directory ./goalpost_inference_graph

/root
/content
/content/drive/My Drive/goalpost_detection_master
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W0728 02:18:41.312906 140549369427840 module_wrapper.py:139] From export_inference_graph.py:98: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0728 02:18:41.319023 140549369427840 module_wrapper.py:139] From /content/drive/My Drive/goalpost_detection_master/TFmodels/research/object_detection/exporter.py:357: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0728 02:18:41.319409 140549369427840 module_wrapper.py:139] From /content/drive/My Drive/goalpost_detection_master/TFmodels/research/objec

##Validating training values

Chegou a hora de testar e ver se o treinamento deu certo ou não, para isso vamos avaliar o desempenho da rede neural em imagens de teste que não estavam presentes nas imagens de treinamento.

###Imports

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from matplotlib.pyplot import imshow

###Env setup

In [ ]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

###Object detection imports
Here are the imports from the object detection module.

In [ ]:
!pip install utils

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/
%cd TFmodels/research/
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

/root
/content
/content/drive/My Drive/goalpost_detection_master
/content/drive/My Drive/goalpost_detection_master/TFmodels/research


##Model preparation 

###Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/
# What model to download.
MODEL_NAME = 'goalpost_inference_graph'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = './' + MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('./object_detection/data', 'goalpost_label_map.pbtxt')

NUM_CLASSES = 1

/root
/content
/content/drive/My Drive/goalpost_detection_master


###Load a (frozen) Tensorflow model into memory.

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

/root
/content
/content/drive/My Drive/goalpost_detection_master


###Loading label map
Label maps map indices to category names, so that when our convolution network predicts `1`, we know that this corresponds to `BB-8` and when it predicts `2`, we know it corresponds to `R2-D2`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine.

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

/root
/content
/content/drive/My Drive/goalpost_detection_master


###Helper code

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

/root
/content
/content/drive/My Drive/goalpost_detection_master


##Detection in images

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/
%matplotlib inline

# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = './test_images_png'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'test{}.png'.format(i)) for i in range(1, 14) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)

## Detection in videos


In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1449984/45929032 bytes (3.2%)4382720/45929032 bytes (9.5%)7929856/45929032 bytes (17.3%)11829248/45929032 bytes (25.8%)15876096/45929032 bytes (34.6%)19816448/45929032 bytes (43.1%)23396352/45929032 bytes (50.9%)27402240/45929032 bytes (59.7%)31096832/45929032 bytes (67.7%)34963456/45929032 bytes (76.1%)38895616/45929032 bytes (84.7%)42901504/45929032 bytes (93.4%)4

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/goalpost_detection_master/test_video

def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # you should return the final output (image with lines are drawn on lanes)
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            image_process = detect_objects(image, sess, detection_graph)
            return image_process

def detect_objects(image_np, sess, detection_graph):
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    # Each box represents a part of the image where a particular object was detected.
    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    # Actual detection.
    (boxes, scores, classes, num_detections) = sess.run(
        [boxes, scores, classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})

    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8)
    return image_np

white_output = 'goalpost_out.mp4'
clip1 = VideoFileClip("goalpost_test_1.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!s
white_clip.write_videofile(white_output, audio=False)

/root
/content
/content/drive/My Drive/goalpost_detection_master/test_video
[MoviePy] >>>> Building video goalpost_out.mp4
[MoviePy] Writing video goalpost_out.mp4


100%|██████████| 1815/1815 [31:48<00:00,  1.05s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: goalpost_out.mp4 

